# Orientation an aspect ratio

In this example, we show how to use inertia matrix of a given labeled object to find its orientation. 

![](./blobs.jpg)

:::{admonition} Required files
:class: important
Before using this notebook, download the image {download}`blobs.jpg <blobs.jpg>`
:::

In [ ]:
%matplotlib widget
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.gridspec import GridSpec
from scipy import ndimage
import pandas as pd
import os

In [ ]:
path = "blobs.jpg"
files = os.listdir("./")
if path in files:
    print("Ok, the file is in {0}".format(files))
else:
    print("The file is not in {0} , retry !".format(files))

In order to explain the concepts of inertia and aspect ratio, we use this magnificient hand drawing:

In [ ]:
im = Image.open(path)
# Resizing image
Nr, Nc = im.size
im = im.resize((Nr // 4, Nc // 4))
# Plot image
plt.figure()
plt.title("Blobs")
plt.ylabel("rows")
plt.xlabel("columns")
plt.imshow(im)
plt.show()

You can use `PIL.Image.split()` class method to extract the color channels.

In [ ]:
R, G, B = im.split()
R = np.array(R)
G = np.array(G)
B = np.array(B)

:::{seealso}
The same can be achieved by using `numpy` slicing, as the image is represented as a 3D array of shape (height, width, channels).

```python
import numpy as np
# Converting the image to a numpy array
image_array = np.array(image)
# Extracting the color channels
R = image_array[:, :, 0]  # Red channel
G = image_array[:, :, 1]  # Green channel
B = image_array[:, :, 2]  # Blue channel
```
:::

## Channel extraction and histogram
We can display every channel separately to see the differences and plot the histogram of pixel values for each channel.

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 3, width_ratios=[1, 1, 1], height_ratios=[1, 1])
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[0, 2])
img_axs = [ax0, ax1, ax2]

ax0.imshow(R, cmap="gray")
ax1.imshow(G, cmap="gray")
ax2.imshow(B, cmap="gray")
ax0.set_title("red channel")
ax1.set_title("green channel")
ax2.set_title("blue channel")
[ax.axis("off") for ax in img_axs]

ax4 = fig.add_subplot(gs[1, :])
ax4.hist(R.flatten(), bins=np.arange(256), histtype="step", color="red", label="red")
ax4.hist(
    G.flatten(), bins=np.arange(256), histtype="step", color="green", label="green"
)
ax4.hist(B.flatten(), bins=np.arange(256), histtype="step", color="blue", label="blue")
ax4.set_title("Histogramms")
ax4.set_xlabel("pixel value")
ax4.set_ylabel("count")
ax4.legend()
plt.show()

## Thresholding and binary image
As we can deduce from the histograms, the thresholding level is obvious and can be set to 50 for all channels. For the sake of simplicity, we will use the blue channel to find the orientation and aspect ratio of the blobs. Here we use [`where()`](https://numpy.org/devdocs/reference/generated/numpy.where.html) from numpy module to create a binary image based on the threshold.

In [ ]:
Bt = np.where(B < 50, 1, 0)
plt.figure()
plt.imshow(Bt, cmap=cm.gray)
plt.colorbar()
plt.show()

## Labelling from binary image
After thresholding, we can use [`scipy.ndimage.label`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.label.html) to label the connected components in the binary image. This will give us a labeled image where each blob has a unique label.

In [ ]:
Btc = ndimage.binary_closing(Bt, structure=np.ones((5, 5)))

Bl, number = ndimage.label(Btc)
plt.figure()
plt.imshow(np.where(Bl != 0, Bl, np.nan), cmap=cm.jet)
plt.show()

In [ ]:
obj = ndimage.find_objects(Bl)
len(obj)

## Inertia matrix of an object

The object represented bellow is stretched in a direction. Let's see how we can use its inertia matrix to determine in which direction and how much it is stretched.

In [ ]:
plt.figure()
plt.imshow(np.array(im)[obj[1]])
plt.show()

The inertia matrix of a 2D object can be defined as follows:

$$
I = 
\begin{bmatrix} 
I_{xx} & -I_{xy} \\
-I_{xy} & I_{yy} \\
\end{bmatrix}
$$

This matrix is symmetric and, as a consequence, it can be diagonalized in a new frame rotated by an angle $\theta$ in the plane. This frame is composed of the two normalized eigenvectors $(\vec e_1, \vec e_2)$ of the matrix. In this frame, the matrix has two eigenvalues $(I_1, I_2)$ ordered so that $I_1 \geq I_2$. Then: 
* $\theta = (\vec x, \vec e_1)$ and,
* The aspect ratio $a = \sqrt{I_1 / I_2}$.

The angle $\theta$ gives the direction of the elongation of the object and $a$ shows how much it is elongated. For example,  if $a == 1$, the object is not elongated whereas if $a=10$ it is 10 times longer in direction 1 than in direction 2 in an inertial point of view.

In [ ]:
data = pd.DataFrame(
    columns=["area", "xg", "yg", "Ixx", "Iyy", "Ixy", "I1", "I2", "theta"]
)
for i in range(len(obj)):
    x, y = np.where(Bl == i + 1)
    xg, yg = x.mean(), y.mean()
    x = x - xg
    y = y - yg
    A = len(x)
    Ixx = (y**2).sum()
    Iyy = (x**2).sum()
    Ixy = (x * y).sum()
    I = np.array([[Ixx, -Ixy], [-Ixy, Iyy]])
    eigvals, eigvecs = np.linalg.eig(I)
    eigvals = abs(eigvals)
    loc = np.argsort(eigvals)[::-1]
    d = eigvecs[loc[0]]
    d *= np.sign(d[0])
    theta = np.degrees(np.arccos(d[1]))
    eigvals = eigvals[loc]
    data.loc[i] = [A, xg, yg, Ixx, Iyy, Ixy, eigvals[0], eigvals[1], theta]
data.sort_values("area", inplace=True, ascending=False)
data["aspect_ratio"] = (data.I1 / data.I2) ** 0.5

data[["area", "theta", "aspect_ratio"]]

In [ ]:
fig = plt.figure()
counter = 1
for i in data.index.values:
    ax = fig.add_subplot(3, 4, counter)
    z = Image.fromarray(np.array(im)[obj[i]])
    z = z.rotate(-data.loc[i, "theta"] + 90, expand=True)
    z = np.array(z)
    plt.imshow(z)
    plt.title(str(i))
    ax.axis("off")
    counter += 1
    # plt.grid()
plt.show()